In [2]:
import pandas as pd
import icecream as ic

transit_cost = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-01-05/transit_cost.csv') 


In [5]:
transit_cost.columns

Index(['e', 'country', 'city', 'line', 'start_year', 'end_year', 'rr',
       'length', 'tunnel_per', 'tunnel', 'stations', 'source1', 'cost',
       'currency', 'year', 'ppp_rate', 'real_cost', 'cost_km_millions',
       'source2', 'reference'],
      dtype='object')

## Rename columns for better understanding

In [8]:
transit_cost.rename(columns={'e':'id', 'rr':'railroad','lenght':'proposed_length','tunnel_per':'tunnel_per_complete',
                            'tunnel':'tunnel_length_complete','cost':'local_cost'}, inplace=True)

In [9]:
transit_cost.head()

,id,country,city,line,start_year,end_year,railroad,length,tunnel_per_complete,tunnel_length_complete,stations,source1,local_cost,currency,year,ppp_rate,real_cost,cost_km_millions,source2,reference
0,7136.0,CA,Vancouver,Broadway,2020,2025,0.0,5.7,87.72%,5.0,6.0,Plan,2830.0,CAD,2018.0,0.84,2377.2,417.052632,Media,https://www.translink.ca/Plans-and-Projects/Ra...
1,7137.0,CA,Toronto,Vaughan,2009,2017,0.0,8.6,100.00%,8.6,6.0,Media,3200.0,CAD,2013.0,0.81,2592,301.395349,Media,https://www.thestar.com/news/gta/transportatio...
2,7138.0,CA,Toronto,Scarborough,2020,2030,0.0,7.8,100.00%,7.8,3.0,Wiki,5500.0,CAD,2018.0,0.84,4620,592.307692,Media,https://urbantoronto.ca/news/2020/03/metrolinx...
3,7139.0,CA,Toronto,Ontario,2020,2030,0.0,15.5,57.00%,8.8,15.0,Plan,8573.0,CAD,2019.0,0.84,7201.32,464.601290,Plan,https://metrolinx.files.wordpress.com/2019/07/...
4,7144.0,CA,Toronto,Yonge to Richmond Hill,2020,2030,0.0,7.4,100.00%,7.4,6.0,Plan,5600.0,CAD,2020.0,0.84,4704,635.675676,Media,https://www.thestar.com/news/gta/2020/06/24/me...


In [10]:
transit_cost.describe()

,id,railroad,length,tunnel_length_complete,stations,local_cost,year,ppp_rate,cost_km_millions
count,537.000000,536.000000,539.000000,512.000000,529.000000,5.370000e+02,537.000000,535.000000,542.000000
mean,7738.763501,0.063433,58.336860,29.376366,13.814745,8.054381e+05,2014.908752,0.660434,232.984386
std,463.234562,0.243968,621.196144,344.037885,13.702870,6.708033e+06,5.637780,0.867256,257.218117
min,7136.000000,0.000000,0.600000,0.000000,0.000000,0.000000e+00,1987.000000,0.000064,7.789626
25%,7403.000000,0.000000,6.500000,3.400000,4.000000,2.289000e+03,2012.000000,0.237900,134.864607
50%,7705.000000,0.000000,15.770000,8.915000,10.000000,1.100000e+04,2016.000000,0.258300,181.250000
75%,7977.000000,0.000000,29.080000,21.525000,20.000000,2.700000e+04,2019.000000,1.000000,241.428571
max,9510.000000,1.000000,12256.983000,7790.782600,128.000000,9.000000e+07,2027.000000,5.000000,3928.571429


In [24]:
country_transit = transit_cost[['country','city']].groupby(by=['country'], axis=0).count()

In [26]:
country_transit.to_csv('country_transit.csv')